In [1]:
import os
import numpy as np
import pandas as pd
import csv 
from python_linq import From

In [6]:
default_path = 'gesture_datas/'
seed = 42
np.random.seed(seed=seed)

In [8]:
def load_data(path):
    all_files = os.listdir(path)
    files_to_load = From(all_files).where(lambda file: file[-3:]=='csv').where(lambda file: file[-5]=='w').toList()
    df_Big = pd.read_csv(path + files_to_load[0], sep=',')
    for file in files_to_load[1:]:
        df_file = pd.read_csv(path + file, sep=',')
        df_Big = df_Big.append(df_file)
    df_Big = df_Big.loc[df_Big['phase']!='Preparação']
    return df_Big

#print(default_path)
dataFrames = load_data(default_path)
print(dataFrames)

           lhx       lhy       lhz       rhx       rhy       rhz        hx  \
0     1.197867  5.502980  2.154376  3.937746  5.555298  2.194409  2.844247   
1     1.192058  5.508098  2.154468  3.937706  5.555610  2.194491  2.839868   
2     1.186308  5.513601  2.154882  3.937459  5.556025  2.194566  2.835107   
3     1.180775  5.519557  2.155479  3.937060  5.556473  2.194650  2.827072   
4     1.175917  5.525742  2.156359  3.936461  5.557368  2.194719  2.819219   
...        ...       ...       ...       ...       ...       ...       ...   
1443  3.924959  5.405544  1.779546  5.087440  5.229737  1.813131  4.388663   
1444  3.851740  5.208477  1.804775  5.084454  5.236107  1.812657  4.392184   
1445  3.851757  5.208529  1.804776  5.082551  5.240685  1.812454  4.393439   
1446  3.851886  5.208885  1.804757  5.077394  5.247313  1.811920  4.393404   
1447  3.851872  5.208958  1.804811  5.075249  5.250179  1.811992  4.393698   

            hy        hz        sx        sy        sz       lw

In [4]:
def truncate(df):
    k = 0
    nb_rows = df.shape[0]
    df_to_fill = pd.DataFrame(columns=df.columns.values.tolist())
    while k < nb_rows:    
        row_considered = df.iloc[[k]]
        phase_considered = row_considered['phase'].values.tolist()[0]
        phase_tmp = phase_considered
        #print(phase_considered)
        #print(phase_tmp)
        i = 0
        while phase_tmp == phase_considered and i<30:
            if k + i + 1 < nb_rows:
                # Verifying if there is a next value
                i += 1
                phase_tmp = df.iloc[[i+k]]['phase'].values.tolist()[0]
            else:
                # End of while
                break
        if i > 7:
            # Add the row considered            
            df_to_fill = df_to_fill.append(row_considered)            
        if k + i + 1 < nb_rows:
            k += i
        else:
            # End of the dataframe
            break
    df_to_fill = df_to_fill.reset_index(drop=True)
    return df_to_fill
df_truncated = truncate(dataFrames)
#print(df_truncated.shape)
#print([df_truncated['phase'].loc[lambda phase: phase==header].count() for header in headers])
#print(df_truncated)

In [5]:
def creat_missing_values(df, nb_rows_to_del=1):
    nb_rows = df.shape[0]
    nb_columns = df.shape[1]
    index_still = np.arange(nb_rows)
    #print(index_still)
    df_copy = df.copy()
    if nb_rows_to_del < nb_rows:
        while nb_rows-len(index_still)<nb_rows_to_del:
            index_del = np.random.randint(len(index_still)-1)
            timestamp_saved = df.iloc[index_del].__getitem__('timestamp')
            #print(timestamp_saved)
            df_copy.iloc[index_del] = [np.nan for i in range(nb_columns)]
            #print(df_copy.iloc[index_del])
            df_copy.loc[index_del, 'timestamp'] = timestamp_saved
            #print(df_copy.iloc[index_del])
            index_still = np.delete(index_still,index_del)
    return df_copy
df_copy = creat_missing_values(df_truncated, 5)
#print(df_truncated.shape)
print(df_copy.head(40))

         lhx       lhy       lhz       rhx       rhy       rhz        hx  \
0   1.197867  5.502980  2.154376  3.937746  5.555298  2.194409  2.844247   
1   1.129344  5.582613  2.184588  3.901658  5.536760  2.178253  2.708008   
2   1.159948  5.519786  2.111937  3.897376  5.569611  2.182385  2.741405   
3   0.975070  5.593150  2.128695  4.126638  4.465309  2.019147  2.702959   
4   1.468093  3.148694  2.046527  2.159844  3.994637  1.936775  2.558908   
5   1.764387  4.022885  1.939738  2.300316  3.978466  1.971898  2.499155   
6   2.391272  3.760352  1.899390  3.643721  4.312732  2.113825  2.656384   
7   1.994954  4.285465  1.864373  3.576632  4.005780  2.023342  2.859714   
8   1.658651  5.507297  2.042797  3.477622  3.948396  2.011934  2.881003   
9   1.333393  5.577484  2.088342  3.452062  3.833898  1.993084  2.886264   
10  1.897931  4.454338  1.954201  3.501980  4.133467  1.984781  2.842583   
11  1.732012  3.012352  2.001570  3.519415  4.289818  1.983916  2.820379   
12  1.962364

In [ ]:
keys = ['lhx', 'lhy', 'lhz', 'rhx', 'rhy', 'rhz', 'hx', 'hy', 'hz', 'sx', 'sy', 'sz', 'lwx', 'lwy', 'lwz', 'rwx', 'rwy', 'rwz', 'timestamp', 'phase']
input_keys = ['lhx', 'lhy', 'lhz', 'rhx', 'rhy', 'rhz', 'hx', 'hy', 'hz', 'sx', 'sy', 'sz', 'lwx', 'lwy', 'lwz', 'rwx', 'rwy', 'rwz']
timestamp_key = 'timestamp'

TODO:
Concatenate all datas,
truncate in this dataset,
make random missing data

Run.py : training / test

In [20]:
timestamp = df_Big['timestamp']
processed_data = {
    'input': [],
    'masking': []
}
n = len(timestamp)